# Обучение на частично размеченной выборке*

Дополнительные материалы к семинару. По мотивам статьи ["Semi-supervised Learning with
Deep Generative Models"](https://arxiv.org/pdf/1406.5298.pdf)

In [ ]:
import sys
import os
import time

import numpy as np
import theano
import theano.tensor as T
import lasagne
import matplotlib.pylab as plt
%matplotlib inline

from utils import load_dataset, iterate_minibatches
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams

Для этого задания мы повысим размерность скрытых компонент, а также случайным образом "выбросим" приблизительно 95% меток классов.

In [ ]:
BATCH_SIZE = 20
HIDDEN_DIM = 16
NUMBER_OF_DIGITS = 10

num_epochs = 40

X_train, y_train, X_val, y_val, X_test, y_test = load_dataset()
present = np.random.rand(X_train.shape[0]) < 0.05

Классы для распределений

In [ ]:
class BinaryVector():
    def __init__(self, logits, rng=None):
        self.rng = rng if rng else RandomStreams(lasagne.random.get_rng().randint(1,2147462579))
        self.logits = logits

    def log_prob(self, x):
        pixelwise_log_probs = (
            x * (self.logits - T.nnet.softplus(self.logits))
            - (1 - x) * T.nnet.softplus(self.logits)
        )
        return T.sum(pixelwise_log_probs, axis=(1, 2, 3))
    
    def sample(self):
        shape = self.logits.shape
        return T.nnet.sigmoid(self.logits) >= self.rng.uniform(shape)

class MultivariateNormalDiag():
    def __init__(self, loc=None, scale=None, rng=None):
        self.rng = rng if rng else RandomStreams(lasagne.random.get_rng().randint(1,2147462579))
        self.loc= loc
        self.scale = scale
    
    def log_prob(self, z):
        normalization_constant = (
            - 0.5 * np.log(2 * np.pi)
            - T.log(self.scale)
        )
        square_term = -0.5 * ((z - self.loc) / self.scale) ** 2
        log_prob_vec = normalization_constant + square_term
        return T.sum(log_prob_vec, axis=1)
    
    def sample(self):
        shape = self.loc.shape
        z = (self.loc + self.scale * self.rng.normal(shape))
        return z

## Вероятностная модель данных

В отличие от вариационного автокодировщика, генеративная модель теперь будет также включать и метки классов $y$:

\begin{align*}
& p(x, y, z) = p(x | y, z) p(z) p(y) \\
& p(y) = Cat(y | \pi), \pi = (1/10, \dots, 1/10) \\
& p(z) = \mathcal N(z | 0, I) \\
& p(x | y, z) = \prod_{i=1}^D p_i(y, z)^{x_i} (1 - p_i(y, z))^{1 - x_i}
\end{align*}

При обучении вариационного автокодировщика максимизируется маргинальное правдоподобие $\log p(x)$ (нижняя оценка на него, если быть точным), а в данном случае мы будем максимизировать $\log p(x,y)$ для объектов с метками и $\log p(x)$ для объектов без метки. Обозначим за $P$ индексы объектов обучающей выборки с метками класса.

Построим нижнюю оценку для

\begin{equation}
L(X, y) = \sum_{i \notin P} \log p(x_i) + \sum_{i \in P} \log p(x_i, y_i)
\end{equation}

Для этого определим следующее вариационное приближение:

\begin{align*}
& q(y, z | x) = q(y | x) q(z | y, x)\\
& \\
& q(y | x) = Cat(y | \pi(x))\\
& q(z | y, x) = \mathcal N(z | \mu_\phi(x, y), \operatorname{diag}\sigma^2(y, x))
\end{align*}

### Оценка для $i \in P$

Случай похож на модель для вариационного автокодировщика

\begin{equation}
\log p(x, y) = \log \mathbb E_{p(z)} p(x, y | z) \geq \mathbb E_{q(z | y, x)} \log \frac{p(x, y|z) p(z)}{q(z | y, x)}
\end{equation}

### Оценка $i \notin P$

\begin{equation}
\log p(x) = \log \mathbb E_{p(y)} \mathbb E_{p(z | y)} \log p(x| z, y)\geq \mathbb E_{q(y | x)} \mathbb E_{q(z | y, x)} \log \frac{p(x, y, z)}{q(z | y, x) q(y | x)}
\end{equation}

### Целевая функия

\begin{equation}
\mathcal L(X, y) = \sum_{i \in P} \mathbb E_{q(z_i | y_i, x_i)} \log \frac{p(x_i, y_i, z_i)}{q(z_i | y_i, x_i)} + \sum_{i \notin P} \mathbb E_{q(y_i | x_i)} \mathbb E_{q(z_i | y_i, x_i)} \log \frac{p(x_i, y_i, z_i)}{q(z_i | y_i, x_i) q(y_i | x_i)}
\end{equation}

Оценку для математического ожидания по $z$ будет получать с помощью *reparametrization trick*.
Пользуясь малым количеством классов, математическое ожидание по $y$ будем вычислять явно.

# Как заставить модель все-таки обучаться?

Максимизация нижней оценки на обоснованность на практике может не приводит к построению хорошей модели вывода $q(y | x)$.

Естественно искать модели $q(y | x)$ среди тех, которые согласуются с размеченными объектами обучающей выборки $(x_i, y_i)$. В статье, в которой была впервые предложена описанная в семинаре модель, с весом $\alpha$ добавляется дополнительное слагаемое к функции потерь:

\begin{equation}
\frac{1}{|P|}\sum_{i \in P} y_i^T \log q(y | x).
\end{equation}

Оно соответствует кросс-энтропии классификатора $q(y|x)$ на размеченных объектах.

### Особенности реализации
В данной реализации мы передаем на вход кодировщика и декодировщика one-hot коды для $y$.

Это находит свое отражение в размерах входов сетей:

In [ ]:
def classifier_mlp(input_x):
    # takes x to produce posterior class assignment probabilities
    l_in = lasagne.layers.InputLayer(shape=(None, 1, 28, 28),
                                     input_var=input_x)
    l_hid1 = lasagne.layers.DenseLayer(
            l_in, num_units=256,
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform(),
            name='cl_hid1')
    l_out = lasagne.layers.DenseLayer(
            l_hid1, num_units=10,
            nonlinearity=lasagne.nonlinearities.softmax,
            name='cl_out')
    return l_out

def vae_encoder_cond(input_xy):
    l_in = lasagne.layers.InputLayer(shape=(None, 28 * 28 + NUMBER_OF_DIGITS),
                                     input_var=input_xy)
    l_hid1 = lasagne.layers.DenseLayer(
        l_in, num_units=256,
        nonlinearity=lasagne.nonlinearities.rectify,
        W=lasagne.init.GlorotUniform(),
        name='e_hid')
    l_out_loc = lasagne.layers.DenseLayer(
        l_hid1, num_units=HIDDEN_DIM,
        nonlinearity=None,
        name='e_mean')
    l_out_scale = lasagne.layers.DenseLayer(
        l_hid1, num_units=HIDDEN_DIM,
        nonlinearity=lasagne.nonlinearities.softplus,
        name='e_scale')
    
    return l_out_loc, l_out_scale
    
    
def vae_decoder_cond(input_zy):
    l_in = lasagne.layers.InputLayer(shape=(None, HIDDEN_DIM + NUMBER_OF_DIGITS),
                                     input_var=input_zy)
    l_hid1 = lasagne.layers.DenseLayer(
            l_in, num_units=256,
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform(),
            name='d_hid1')
    l_out = lasagne.layers.DenseLayer(
            l_hid1, num_units=28 * 28,
            nonlinearity=None,
            name='d_out')
    l_out = lasagne.layers.ReshapeLayer(l_out, shape=(-1, 1, 28, 28))
    return l_out

При обучении мы будем вычислять выходы нейросети на всех возможных значениях $y$, все они нужны в 95% случаев для подсчета нижней оценки на обоснованность. Для этого здесь написаны две вспомонательные функции:

In [ ]:
input_x = T.tensor4('input_x')
input_y = T.ivector('input_y')
input_p = T.bvector('input_present')

def add_all_possible_labels(input_x):
    # создает десять копий объекта из батча и приписывает к каждой из них код для y
    input_x = T.reshape(input_x, newshape=(BATCH_SIZE, -1))
    input_x = T.repeat(input_x, repeats=NUMBER_OF_DIGITS, axis=0)
    input_y = T.repeat(T.eye(NUMBER_OF_DIGITS), repeats=BATCH_SIZE, axis=0)
    input_xy = T.concatenate([input_x, input_y], axis=1)
    return input_xy

def add_corresponding_labels(input_z):
    # приписывает код n % 10 (остаток деления) для n объекта в батче
    input_y = T.repeat(T.eye(NUMBER_OF_DIGITS), repeats=BATCH_SIZE, axis=0)
    input_zy = T.concatenate([input_z, input_y], axis=1)
    return input_zy

Модель вывода

In [ ]:
input_xy = add_all_possible_labels(input_x)
encoder_mean, encoder_scale = vae_encoder_cond(input_xy)
qz_xy = MultivariateNormalDiag(
    lasagne.layers.get_output(encoder_mean), 
    lasagne.layers.get_output(encoder_scale)
)

input_zy = add_corresponding_labels(qz_xy.sample())


Генеративная модель

In [ ]:
decoder_logits = vae_decoder_cond(input_zy)
pz = MultivariateNormalDiag(T.zeros((NUMBER_OF_DIGITS * BATCH_SIZE, HIDDEN_DIM)),
                            T.ones((NUMBER_OF_DIGITS * BATCH_SIZE, HIDDEN_DIM)))
# здесь мы не стали реализовывать отдельный класс
p_y = -np.log(NUMBER_OF_DIGITS * np.ones([BATCH_SIZE * NUMBER_OF_DIGITS]))

px_zy = BinaryVector(
    lasagne.layers.get_output(decoder_logits)
)

classifier = classifier_mlp(input_x)
qy_x_probs = lasagne.layers.get_output(classifier)

Функция потерь

In [ ]:
alpha = 5.

elbo_vec = (+ p_y
            + px_zy.log_prob(T.repeat(input_x, repeats=NUMBER_OF_DIGITS, axis=0))
            + pz.log_prob(qz_xy.sample())
            - qz_xy.log_prob(qz_xy.sample()))
elbo_vec = T.reshape(elbo_vec, newshape=(BATCH_SIZE, NUMBER_OF_DIGITS))

elbo_vec = (
    input_p * T.sum(elbo_vec * lasagne.utils.one_hot(input_y, m=NUMBER_OF_DIGITS), axis=1)
    + (1 - input_p) * T.sum(qy_x_probs * elbo_vec - qy_x_probs * T.log(qy_x_probs), axis=1)
)

loss = T.mean(elbo_vec - alpha * input_p * lasagne.objectives.categorical_crossentropy(qy_x_probs, input_y))

In [ ]:
params = lasagne.layers.get_all_params(
    [encoder_mean, encoder_scale, decoder_logits, classifier]
)
updates = lasagne.updates.adam(-loss, params)

train_fn = theano.function([input_x, input_y, input_p], loss, updates=updates)
accuracy = theano.function(
    [input_x, input_y],
    T.mean(T.eq(T.argmax(qy_x_probs, axis=1), input_y), dtype=theano.config.floatX)
)

In [ ]:
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, present=present, batchsize=BATCH_SIZE):
        inputs, targets, batch_present = batch
        inputs = np.random.rand(*inputs.shape) < inputs
        train_err += train_fn(inputs, targets, batch_present)
        train_batches += 1
    
    test_accuracy = 0
    test_batches = 0
    for batch in iterate_minibatches(X_test, y_test, batchsize=BATCH_SIZE, shuffle=False):
        inputs, targets = batch
        inputs = np.random.rand(*inputs.shape) < inputs
        test_accuracy += accuracy(inputs, targets)
        test_batches += 1
    
    print("Epoch {} of {} took {:.3f}s".format(
          epoch + 1, num_epochs, time.time() - start_time))
    print("Train elbo {}".format(train_err/train_batches))
    print("Test accuracy {}".format(test_accuracy/test_batches))

# Задание*

Ниже приведен код, генерирующий случайные цифры из заданного класса.

Эксперементируя с архитектурами сети и параметрами модели, попробуйте обучить модель, для которой успешно выполняется это сэмплирование (см. эксперементальные результаты статьи https://arxiv.org/pdf/1406.5298.pdf)

In [ ]:
input_z = T.matrix('input_z')

decode_a_code = theano.function(
    [input_z],
    lasagne.layers.get_output(decoder_logits, input_z),
)

In [ ]:
digit_to_draw = 4

z_samples = np.random.randn(64, HIDDEN_DIM)
y_samples = np.zeros((64, NUMBER_OF_DIGITS))
y_samples[:, digit_to_draw] = 1
zy_samples = np.concatenate([z_samples, y_samples], axis=1)

decoded_images = decode_a_code(zy_samples)

fig, axes = plt.subplots(8, 8, figsize=(8, 8),
    subplot_kw={'xticks': [], 'yticks': []}
)
fig.subplots_adjust(hspace=0.04, wspace=0.02)

for ax, i in zip(axes.flat, range(64)):
    ax.imshow(decoded_images[i].reshape((28, 28)), cmap='gray')